# The problem

Consider the following scenario: due to my obsession with planning, I set up a goal for the nightlife in Berlin: target percentages for each of several local bars and clubs to know the proportion of nights I want to spend there:

+ SchwuZ: 28%
+ KitKat: 22%
+ Grosse Freiheit: 15%
+ Prinzknecht: 25%
+ Berghain: 10%

However after 24 weeks, the actual shares of my visits are as follows:

+ SchwuZ: 16.2%
+ KitKat: 23.5%
+ Grosse Freiheit: 14.8%
+ Prinzknecht: 27.6%
+ Berghain: 17.9%

With only 8 weeks left until I move out of the city, I need to establish new targets for the remaining weeks to approach the original goals as closely as possible. However, it becomes impossible to achieve exact alignment when the difference is substantial, as there may not be enough available nights to compensate. Formulating a set of linear equations results in values outside the interval [0%, 100%].

The best approach is to **optimize the numbers** to get as close as possible to the original targets while respecting the boundary conditions. This optimization process can be accomplished using **the 'minimize' module of the 'scipy' library**.

# The mathematics

Let $t_{1}, ..., t_{5}$ be targets for individual clubs, $cs_{1}, ..., cs_{5}$ current shares and $nt_{1}, ..., nt_{5}$ unknown new targets. The total number of weeks $N_{tot} = 32$, the weeks passed $N_{passed} = 24$. Then for each club holds

$$N_{tot} \cdot t_{i} = N_{passed} \cdot cs_{i} + (N_{tot} - N_{passed}) \cdot nt_{i}$$

and thus

$$nt_{i} = \frac{N_{passed}}{N_{tot} - N_{passed}} \cdot (t_{i} - cs_{i}) + t_{i}.$$

$nt_{i}$ can have values outside the interval [0%, 100%].

# Optimization

### Import

Import modules and libraries - 'numpy' for easy work with arrays and 'minimize' from scipy for the optimization procedure.

In [1]:
import numpy as np   # for an easier work with arrays
from scipy.optimize import minimize   # a module for finding the minimum of given function

### Functions to optimize

3 functions will be handy in this case.
* The function setting up new targets based on the goal and current state,
* a function be minimized - the difference between achievable optimized new targets and their mathematical representation ('objective function'),
* the final shares to compare with objectives.

In [2]:
# equation for calculating new targets
def calculate_new_percentages(t_i, cs_i, N_tot, N_passed):
    return (N_passed/(N_tot - N_passed))*(np.array(t_i) - np.array(cs_i)) + np.array(t_i)

# objective function using the equation above
def objective_function(nt_i, t_i, cs_i, N_tot, N_passed):
    return np.sum((nt_i - calculate_new_percentages(t_i, cs_i, N_tot, N_passed))**2)

# final shares
def final_share(nt_i, cs_i, N_tot, N_passed):
    return (np.array(cs_i)*N_passed + np.array(nt_i)*(N_tot - N_passed))/N_tot

### Calculation

The 'optimize' package needs an initial guess to start numerically calculating the optimized values. I tried with 5, 50 and 95 and the result was pretty much the same.

In [3]:
# initial guess for nt_i
initial_guess = [50 for _ in range(5)]

Two kind of conditions can be set: bounds and constraints. Bounds are boundaries for the individual variable values, in my case all percentages must be between 0 and 100. Constraints are aditional conditions, here all the new targets must sum up to 100 %. The type of constraint is equality and the function specified says that the sum of new targets minus 100 % should be equal to zero.

In [4]:
# bounds
bounds_perc = [(0, 100) for _ in range(5)]

# constraint
constraint_sum = {'type': 'eq', 'fun': lambda x: np.sum(x) - 100}

Now I specify the numerical input - original targets, current shares between clubs, the number of weeks in total and passed. With this set of numbers, precise target cannot be reached (try calculating using the 'calculate_new_percentages' function and skipping the bounds).

In [5]:
# numerical input
t_i = [28, 22, 15, 25, 10] # original targets
cs_i = [16.2, 23.5, 14.8, 27.6, 17.9] # current shares
N_tot = 32 # total number of weeks
N_passed = 24 # number of weeks passed

Perform the optimization and extract the results.

In [6]:
# optimization
result = minimize(objective_function, initial_guess, args=(t_i, cs_i, N_tot, N_passed), constraints=constraint_sum,
                  bounds=bounds_perc)

# extract results
nt_i = result.x

Print the results and see that the condition was met.

In [7]:
print('Optimized values for nt_i:', np.round(nt_i, 2))
print('The sum of new targets:', np.sum(nt_i))

Optimized values for nt_i: [59.97 14.08 12.18 13.78  0.  ]
The sum of new targets: 100.00000000000001


### Final comparison

Calculate the final shares and the results with respect to original targets.

In [8]:
# final shares and difference
fs_i = final_share(nt_i, cs_i, N_tot, N_passed)
delta = fs_i - t_i

In [9]:
# print the check
fs_i = final_share(nt_i, cs_i, N_tot, N_passed)
print('Final shares fs_i:', fs_i)
print('Difference fs_i - t_i:', delta)

Final shares fs_i: [27.14374972 21.14375008 14.14375008 24.14375012 13.425     ]
Difference fs_i - t_i: [-0.85625028 -0.85624992 -0.85624992 -0.85624988  3.425     ]


Based on the numerical input, we can see different results, for the input from this example, the first four clubs differ by less than one percent and Berghain by about 3.4 %. This is the closest I can approach the original targets.